LETS GET GERF AND YOLO WORKING!!!!

First we're going to download ultralytics repository which contains the yolo repository.

In [1]:
!git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...
remote: Enumerating objects: 76852, done.
remote: Counting objects: 100% (615/615), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 76852 (delta 518), reused 381 (delta 373), pack-reused 76237 (from 3)
Receiving objects: 100% (76852/76852), 41.11 MiB | 25.99 MiB/s, done.
Resolving deltas: 100% (57714/57714), done.


Next, we're going to copy over the custom files to the ultralytics repository
Note, the yolo11_custom.yaml file contains the architecture that incorporates GERF model. We'll leave it inside the custom directory on purpose.

In [2]:
!cp custom/__init__.py ultralytics/ultralytics/nn/modules
!cp custom/gerf_bra.py ultralytics/ultralytics/nn/modules
!cp custom/tasks.py ultralytics/ultralytics/nn

Now, we need to export the python path to include this repository to use with our python code.
Ideally, we would install with "pip install -e ." and the ultralytics toml file but for some reason,
it would do a clean install and ignore my changes
1. First we'll collect your current path
2. Then we'll check what's on PYTHONPATH
3. Add the outputs of your current path from 1 and the PYTHONPATH to the environment variable
4. Finally, we'll verify the variable was set

In [3]:
!echo "Check where the ultralytics repo is stored"
!pwd

Check where the ultralytics repo is stored
/content


In [4]:
!echo "Check current python path. Should be empty at every new session"
!echo $PYTHONPATH

Check current python path. Should be empty at every new session
/env/python


In [5]:
!echo "Now let's set the path. I got the below path from the pwd command and added in ultralytics"
%env PYTHONPATH=/content/ultralytics:/env/python

Now let's set the path. I got the below path from the pwd command and added in ultralytics
env: PYTHONPATH=/content/ultralytics:/env/python


In [6]:
!echo "Check current python path. Should be updated now"
!echo $PYTHONPATH

Check current python path. Should be updated now
/content/ultralytics:/env/python


Now we can test out if the GERF additions are ready to use
The first cell tests the CLI
The second cell actually imports the custom yolo repository for python notebook usage

In [7]:
!python -c "from ultralytics import YOLO; from ultralytics.nn.modules import GERF_BRA_Simple; print('All good!')"

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
All good!


Now that we confirmed the module is ready, let's go ahead and create our yolo11 gerf model.
The model is defined in custom/yolo11_custom.yaml file. One big thing to note, the argument for
the gerf block is indicated by the \[-1, 1, GERF_BRA, \[512\]\] line. There's two GERF_BRA lines.
the main thing to know from this is that the \[512\] is an argument field to GERF_BRA module.
The module itself does not need an input but the way that ultralytics reads the yaml file, it assumes
each block has an input. So python crashes without this quirk. GERF is dynamic so it changes per input.

In [9]:
!python mainGERF.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Training on GPU
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco128.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=custom/yolo11_custom.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_gerf_gpu, nbs=64, nms=False, opset=None, optimize=False, optimize

In [11]:
!python inferenceGERF.py
!mkdir -p resultsGerf
!mv *.jpg resultsGerf
!echo "Only trained on subset of COCO, so most likely won't show anything in results"

Processing: datasets/coco128/images/train2017/000000000009.jpg

image 1/1 /content/datasets/coco128/images/train2017/000000000009.jpg: 480x640 24 persons, 1 car, 53.7ms
Speed: 1.8ms preprocess, 53.7ms inference, 11.5ms postprocess per image at shape (1, 3, 480, 640)
Processing: datasets/coco128/images/train2017/000000000025.jpg

image 1/1 /content/datasets/coco128/images/train2017/000000000025.jpg: 448x640 (no detections), 49.4ms
Speed: 1.6ms preprocess, 49.4ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)
Processing: datasets/coco128/images/train2017/000000000030.jpg

image 1/1 /content/datasets/coco128/images/train2017/000000000030.jpg: 448x640 (no detections), 8.4ms
Speed: 1.5ms preprocess, 8.4ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)
Inference complete. Results saved.
Only trained on subset of COCO, so most likely won't show anything in results


In [12]:
!python mainYolo11.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco128.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=custom/yolo11.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_gpu, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, p

In [16]:
!python inferenceYOLO.py
!mkdir -p resultsYolo
!mv *.jpg resultsYolo
!echo "Only trained on subset of COCO, so most likely won't show anything in results"

Processing: datasets/coco128/images/train2017/000000000009.jpg

image 1/1 /content/datasets/coco128/images/train2017/000000000009.jpg: 480x640 8 persons, 1 motorcycle, 3 benchs, 1 snowboard, 1 sports ball, 1 hot dog, 1 couch, 1 potted plant, 4 teddy bears, 82.9ms
Speed: 2.7ms preprocess, 82.9ms inference, 13.6ms postprocess per image at shape (1, 3, 480, 640)
Processing: datasets/coco128/images/train2017/000000000025.jpg

image 1/1 /content/datasets/coco128/images/train2017/000000000025.jpg: 448x640 (no detections), 86.0ms
Speed: 1.9ms preprocess, 86.0ms inference, 0.9ms postprocess per image at shape (1, 3, 448, 640)
Processing: datasets/coco128/images/train2017/000000000030.jpg

image 1/1 /content/datasets/coco128/images/train2017/000000000030.jpg: 448x640 (no detections), 12.2ms
Speed: 1.8ms preprocess, 12.2ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)
Inference complete. Results saved.
Only trained on subset of COCO, so most likely won't show anything in resul

In [17]:
!python mainGERF_AllCoco.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Training on GPU
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=custom/yolo11_custom.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_gerf_allcoco_gpu, nbs=64, nms=False, opset=None, optimize=False, opt

In [ ]:
!python mainYolo11_AllCoco.py.py